
### Exercise

1) Make a new notebook and set up an adsorption configuration where the N<sub>2</sub> molecule is lying down with the center of mass above a three-fold hollow site as shown below. Use an adsorption height of 1.7 Å.

<img src="N2Ru_hollow.png">

Remember that you can read in the `traj` files you have saved, so you don't need to optimise the surface again.

View the combined system before you optimize the structure to ensure that you created what you intended.


In [ ]:
from ase import Atoms
from gpaw import GPAW, PW
from ase.constraints import FixAtoms
from ase.optimize import QuasiNewton
from ase.build import fcc111, hcp0001
import numpy as np
from ase.visualize import view
from ase.io import read, write
import time

a_Ru = 2.704  # PBE value from OQMD.org; expt value is 2.706
slab = hcp0001('Ru', a=a_Ru, size=(2, 2, 2), vacuum=5.0)

# Other metals are possible, for example Rhodium
# Rhodium is FCC so you should use fcc111(...) to set up the system (same arguments).
# Remember to set the FCC lattice constant, get it from OQMD.org.

# a_Rh = 3.793
# slab = fcc111('Rh', a=a_Rh, size=(2, 2, 2), vacuum=5.0)

calc = GPAW(xc='PBE',
            mode=PW(350),
            kpts={'size': (4, 4, 1), 'gamma': True},
            convergence={'eigenstates': 1e-6})
slab.calc = calc

z = slab.positions[:, 2]
constraint = FixAtoms(mask=(z < z.min() + 1.0))
slab.set_constraint(constraint)
dyn = QuasiNewton(slab, trajectory='Ru.traj')
t = time.time()
dyn.run(fmax=0.05)
#print(f'Wall time: {(time.time() - t) / 60} min.')

slabN2= read('.traj')
#iter0 = read('Ru.traj', index=0)
#print('Energy: ', iter0.get_potential_energy())
#print('Forces: ', iter0.get_forces())

e_slab = slab.get_potential_energy()
#print(e_slab)

d = 1.10
molecule = Atoms('2N', positions=[(0., 0., 0.), (0., 0., d)])
molecule.set_cell(slab.get_cell())
molecule.center()
calc_mol = GPAW(xc='PBE', mode=PW(350))
molecule.calc = calc_mol
dyn2 = QuasiNewton(molecule, trajectory='N2.traj')
dyn2.run(fmax=0.05)
e_N2 = molecule.get_potential_energy()

#d_N2 = molecule.get_distance(0, 1)
#print(d_N2)

h = 1.9  # guess at the binding height
d = 1.2  # guess at the binding distance
slab.positions[4, 2] += 0.2  # pre-relax the binding metal atom.

molecule = Atoms('2N', positions=[(0, 0, 0), (0, 0, d)])
p = slab.get_positions()[4]
molecule.translate(p + (0, 0, h))
slabN2 = slab + molecule
constraint = FixAtoms(mask=(z < z.min() + 1.0))
slabN2.set_constraint(constraint)
#view(slabN2)

slabN2.calc = calc
dyn = QuasiNewton(slabN2, trajectory='N2Ru-top.traj', maxstep=0.02)
t = time.time()
dyn.run(fmax=0.05)
print(f'Wall time: {(time.time() - t) / 60} min.')

d_N2_ads2 = slabN2.get_distance(4,8)
print(d_N2_ads2)

In [ ]:
slab = read('Ru.traj')
view(slab)


Note that when viewing the structure, you can find the index of the individual atoms in the ``slab`` object by clicking on them.

You might also find the [`get_center_of_mass()`](https://wiki.fysik.dtu.dk/ase/ase/atoms.html#ase.Atoms.get_center_of_mass) and [`rotate()`](https://wiki.fysik.dtu.dk/ase/ase/atoms.html#ase.Atoms.rotate) methods useful.



Now you should optimize the structure as you did before with the N<sub>2</sub> molecule standing.  The calculation will probably bee too long to run interactively in a Notebook.  Prototype it here, then interrupt the calculation and copy-paste the relevant cells into a script.

Check the number of irreducible k-points and then submit the job as a batch job running on that number of CPU cores.



3) Make a configuration where two N atoms are adsorbed in hollow sites on the surface as shown below

<img src='2NadsRu.png'>

Note that here the two N atoms sit on next-nearest hollow sites.  An alternative would be to have them on nearest neighbor sites.  If you feel energetic you could investigate that as well.  Also, there are two different kinds of hollow sites, they are not completely equivalent!



Optimise the structure and get the final energy. Is it favourable to dissociate N<sub>2</sub> on the surface? What is the N-N distance now? What does that mean for catalysis?
